In [1]:
import grpc
from tensorflow import make_tensor_proto
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_log_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
from tensorflow_serving.apis import get_model_metadata_pb2

2022-05-19 07:18:25.479545: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 07:18:25.479585: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import cv2
import numpy as np
from PIL import Image
from patchify import patchify

PATCH_SIZE = 480

def patch_image(image):

    instances = []

    if type(image) == str:
        image = cv2.imread(image)
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    size_x = (image.shape[1] // PATCH_SIZE) * PATCH_SIZE  # get width to nearest size divisible by patch size
    size_y = (image.shape[0] // PATCH_SIZE) * PATCH_SIZE  # get height to nearest size divisible by patch size

    image = Image.fromarray(image)

    # Crop original image to size divisible by patch size from top left corner
    image = np.array(image.crop((0, 0, size_x, size_y)))

    # Extract patches from each image, step=patch_size means no overlap
    patch_img = patchify(image, (PATCH_SIZE, PATCH_SIZE, 3), step=PATCH_SIZE)

    # iterate over vertical patch axis
    for j in range(patch_img.shape[0]):
        # iterate over horizontal patch axis
        for k in range(patch_img.shape[1]):
            # patches are located like a grid. use (j, k) indices to extract single patched image
            single_patch_img = patch_img[j, k]

            # Drop extra extra dimension from patchify
            instances.append(np.squeeze(single_patch_img))

    return instances, (int(size_x/PATCH_SIZE), int(size_y/PATCH_SIZE))

In [3]:
def depatchify(patches_arr, size, p_size=PATCH_SIZE):
    patches_np = np.array(patches_arr)

    w, h = size

    reshaped = patches_np.reshape(w, h, p_size, p_size, 3)

    bag = []

    for subindex in range(reshaped.shape[0]):
        line = np.concatenate(reshaped[subindex],axis=1)
        bag.append(line)

    image_to_reshape = np.array(bag)



    return image_to_reshape.reshape(w*p_size, h*p_size, 3)

In [4]:
from enum import Enum
class MaskColorMap(Enum):
    Urban = (0, 255, 255) # Cyan
    Agriculture = (255, 255, 0) # Amarillo
    Rangeland = (255, 0, 255) # Morado
    Forest = (0, 255, 0) # Verde
    Water = (0, 0, 255) # Azul
    Barren = (255, 255, 255) # Blanco
    Uknown = (0,0,0) # Negro

In [5]:
from tensorflow.keras.backend import flatten, sum
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

def jaccard_index(y_true, y_pred):
    y_true_f = flatten(y_true)
    y_pred_f = flatten(y_pred)
    intersection = sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (sum(y_true_f) + sum(y_pred_f) - intersection + 1.0)

In [6]:
def rgb_encode_mask(mask):
    # initialize rgb image with equal spatial resolution
    rgb_encode_image = np.zeros((mask.shape[0], mask.shape[1], 3))

    # iterate over MaskColorMap
    for j, cls in enumerate(MaskColorMap):
        # convert single integer channel to RGB channels
        rgb_encode_image[(mask == j)] = np.array(cls.value) / 255.
    return rgb_encode_image

In [7]:
def visualize(image):
    img = Image.fromarray(image)
    img.show()

In [8]:
from keras import backend as K
from keras.models import load_model
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

# Function to export Keras model to Protocol Buffer format
# Inputs:
#       path_to_h5: Path to Keras h5 model
#       export_path: Path to store Protocol Buffer model

def export_h5_to_pb(path_to_h5, export_path):

    # Set the learning phase to Test since the model is already trained.
    K.set_learning_phase(0)

    # Load the Keras model
    keras_model = load_model(path_to_h5, custom_objects={'jaccard_index': jaccard_index})

    # Build the Protocol Buffer SavedModel at 'export_path'
    builder = saved_model_builder.SavedModelBuilder(export_path)

    # Create prediction signature to be used by TensorFlow Serving Predict API
    signature = predict_signature_def(inputs={"images": keras_model.input},
                                      outputs={"scores": keras_model.output})

    with K.get_session() as sess:
        # Save the meta graph and the variables
        builder.add_meta_graph_and_variables(sess=sess, tags=[tag_constants.SERVING],
                                         signature_def_map={"predict": signature})

    builder.save()

In [16]:
from tensorflow.keras.models import load_model
import tensorflow as tf
if tf.executing_eagerly():
   tf.compat.v1.disable_eager_execution()


MODEL_FILE = 'new_ecrop_model.h5'

model = load_model(MODEL_FILE, custom_objects={'jaccard_index': jaccard_index})

2022-05-19 07:24:30.373376: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-19 07:24:30.374015: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /media/DATA1/shared/PBL6/env/lib/python3.8/site-packages/cv2/../../lib64:
2022-05-19 07:24:30.374305: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /media/DATA1/shared/PBL6/env/lib/python3.8/site-packages/cv2/../../lib64:
2022-05-19 07:24:30.374823: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.1

In [16]:
export_h5_to_pb('new_ecrop_model.h5', 'ecrop-main-serving')

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ecrop-main-serving/saved_model.pb


In [9]:
channel = grpc.insecure_channel('localhost:8500')

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

MODEL = 'ecrop-main-serving'
VERSION = 1

INPUT_LAYER = 'input_1'
OUTPUT_LAYER = 'conv2d_18'

request = predict_pb2.PredictRequest()
request.model_spec.name = MODEL
request.model_spec.version.value = VERSION


In [10]:
patches, size = patch_image('input/37586_sat.jpg')

img = patches[0]

In [11]:
sendable = np.expand_dims(img, 0)

request.inputs[INPUT_LAYER].CopyFrom(make_tensor_proto(sendable, shape=sendable.shape, dtype=float))

In [12]:
sendable.shape

(1, 480, 480, 3)

In [13]:
sendable.dtype

dtype('uint8')

In [17]:
model.predict(sendable)

/media/DATA1/shared/PBL6/env/lib/python3.8/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


array([[[[1.64815467e-02, 7.53952742e-01, 5.91568016e-02, ...,
          2.09163851e-03, 1.64591119e-01, 8.83900702e-06],
         [8.16725660e-03, 8.25182021e-01, 3.62326354e-02, ...,
          2.42826319e-03, 1.25607163e-01, 5.63292951e-06],
         [6.70527248e-03, 8.04976225e-01, 2.95364335e-02, ...,
          1.75316550e-03, 1.54991075e-01, 4.49290155e-06],
         ...,
         [3.05010881e-02, 4.54666913e-01, 8.47352594e-02, ...,
          2.99233827e-03, 4.19645905e-01, 5.00178903e-05],
         [3.16225924e-02, 4.86861885e-01, 1.34517089e-01, ...,
          1.80737569e-03, 3.37441683e-01, 1.37286006e-05],
         [3.36461999e-02, 4.83076274e-01, 1.57646641e-01, ...,
          1.69270823e-03, 3.14190060e-01, 1.07841106e-05]],

        [[1.16860922e-02, 7.91635096e-01, 4.17574830e-02, ...,
          2.12590024e-03, 1.50582671e-01, 9.97508960e-06],
         [5.77112474e-03, 8.68918955e-01, 2.60704756e-02, ...,
          2.12276541e-03, 9.58790928e-02, 4.49094432e-06],
        

In [14]:
result = stub.Predict(request, 10)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INVALID_ARGUMENT
	details = "2 root error(s) found.
  (0) INVALID_ARGUMENT: Conv2DSlowBackpropInput: Size of out_backprop doesn't match computed: actual = 30, computed = 10 spatial_dim: 2 input: 20 filter: 2 output: 30 stride: 2 dilation: 1
	 [[{{function_node __inference__wrapped_model_1708}}{{node model/conv2d_transpose/conv2d_transpose}}]]
	 [[StatefulPartitionedCall/_315]]
  (1) INVALID_ARGUMENT: Conv2DSlowBackpropInput: Size of out_backprop doesn't match computed: actual = 30, computed = 10 spatial_dim: 2 input: 20 filter: 2 output: 30 stride: 2 dilation: 1
	 [[{{function_node __inference__wrapped_model_1708}}{{node model/conv2d_transpose/conv2d_transpose}}]]
0 successful operations.
0 derived errors ignored."
	debug_error_string = "{"created":"@1652944743.675782258","description":"Error received from peer ipv6:[::1]:8500","file":"src/core/lib/surface/call.cc","file_line":952,"grpc_message":"2 

In [58]:
from tensorflow.keras.models import load_model

model = load_model('ecrop-main-serving', custom_objects={'jaccard_index': jaccard_index})


In [64]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 480, 480, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 480, 480, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 480, 480, 16  448         ['rescaling[0][0]']              
                                )                                                                 
                                                                                              

In [90]:
model.save('ecrop-main-serving', save_format='tf',  )

INFO:tensorflow:Assets written to: ecrop-main-serving/assets


In [94]:
out = model.predict(sendable.astype(np.uint8))